# **SETUP**

In [1]:
import torch
import os
import sys

import pickle
import itertools
from tqdm import tqdm

from contextlib import redirect_stdout, redirect_stderr

current_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(current_dir, "../src/")))
from engine import run_vit_experiment, run_cnn_experiment
from config import VIT_MODEL_CONFIG_MAP, CNN_MODEL_CONFIG_MAP, OPTIMIZER_CONFIG_MAP
from utils import _DevNull

print("CUDA available:", torch.cuda.is_available())
print("Number of GPUs:", torch.cuda.device_count())
for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

/home/hoai-linh.dao/Envs/brats-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CUDA available: True
Number of GPUs: 2
GPU 0: NVIDIA L40
GPU 1: NVIDIA L40


In [2]:
DEVICE = "cuda:0"
ROOT = "/home/hoai-linh.dao/Works/BraTS/results/self-experiments/cnns-Figshare-x10"
CHECKPOINT_FILE = os.path.join(ROOT, "checkpoint.pkl") # for monitorring progress
SUMMARY_FILE = os.path.join(ROOT, "summary.txt")
DTS_DIR = "/home/hoai-linh.dao/Works/BraTS/dts/Figshare_x10" 
DATALOADER_TYPE = "flat" # Figshare | Figshare_X10 : "flat"; BT_4C | MRI-Scan : "split_folder"; Br35H: "binary_class"
if os.path.exists(CHECKPOINT_FILE):
    with open(CHECKPOINT_FILE, "rb") as f:
        all_results = pickle.load(f)
else:
    all_results = {}
# # Config for ViTs
# model_patch_sizes = ["B16", "L16", "B32", "L32"]
# img_nets = ["1K", "21K"]
# case_nums = ["1", "2", "3"]
# optimizer_names = ["adam"]

# Config for CNNs
model_names = ["VGG16", "ResNet50", "GoogLeNet", "MobileNetV2", "DenseNet121", "ConvNeXt_Large", "EfficientNetB0"]
case_nums = ["5"]
optimizer_names = ["adam"]


# **BENCH TEST**

In [3]:
# combinations = list(itertools.product(model_patch_sizes, img_nets, case_nums, optimizer_names))

# remaining_combinations = [
#     (patch_size, img_net, case_num, optimizer)
#     for patch_size, img_net, case_num, optimizer in combinations
#     if f"{patch_size}_{img_net}_case{case_num}_{optimizer}" not in all_results
# ]

# pbar = tqdm(
#     remaining_combinations,
#     total=len(remaining_combinations),
#     desc="Running experiments"
# )

# for patch_size, img_net, case_num, optimizer in pbar:
#     config_key = f"{patch_size}_{img_net}_case{case_num}_{optimizer}"
#     experiment_name = f"{patch_size}_{img_net}_case{case_num}"
#     log_path = os.path.join(
#         ROOT, "logs", optimizer, f"{experiment_name}.txt"
#     )

#     pbar.set_description(f"{config_key}")
#     pbar.set_postfix(log=log_path)

#     try:
#         with redirect_stdout(_DevNull()), redirect_stderr(_DevNull()):
#             result = run_vit_experiment(
#                 model_patch_size     = patch_size,
#                 img_net              = img_net,
#                 case_num             = case_num,
#                 optimizer_name       = optimizer,
#                 seed                 = 42,
#                 dataset_dir          = DTS_DIR,
#                 num_classes          = 3,
#                 device               = DEVICE,
#                 experiment_root      = ROOT,
#                 model_config_map     = VIT_MODEL_CONFIG_MAP,
#                 optimizer_config_map = OPTIMIZER_CONFIG_MAP,
#                 dataloader_type      = DATALOADER_TYPE
#             )

#         all_results[config_key] = result
#         with open(CHECKPOINT_FILE, "wb") as f:
#             pickle.dump(all_results, f)

#     except Exception as e:
#         tqdm.write(f"[FAIL] {config_key}: {e}")

# with open(SUMMARY_FILE, "w", encoding="utf-8") as f:
#     for config, result in all_results.items():
#         weight_path = result.get('weights_path', 'No information for weight')
#         f.write(f"{config}: Weights saved at {weight_path}\n")


In [4]:
combinations = list(itertools.product(model_names, case_nums, optimizer_names))

remaining_combinations = [
    (model_names, case_num, optimizer)
    for model_names, case_num, optimizer in combinations
    if f"{model_names}_case{case_num}_{optimizer}" not in all_results
]

pbar = tqdm(
    remaining_combinations,
    total=len(remaining_combinations),
    desc="Running experiments"
)

for model_name, case_num, optimizer in pbar:
    config_key = f"{model_name}_case{case_num}_{optimizer}"
    experiment_name = f"{model_name}_case{case_num}"
    log_path = os.path.join(
        ROOT, "logs", optimizer, f"{experiment_name}.txt"
    )

    pbar.set_description(f"{config_key}")
    pbar.set_postfix(log=log_path)

    try:
        with redirect_stdout(_DevNull()), redirect_stderr(_DevNull()):
            result = run_cnn_experiment(
                model_name           = model_name,
                case_num             = case_num,
                optimizer_name       = optimizer,
                seed                 = 42,
                dataset_dir          = DTS_DIR,
                num_classes          = 3,
                device               = DEVICE,
                experiment_root      = ROOT,
                model_config_map     = CNN_MODEL_CONFIG_MAP,
                optimizer_config_map = OPTIMIZER_CONFIG_MAP,
                dataloader_type      = DATALOADER_TYPE
            )

        all_results[config_key] = result
        with open(CHECKPOINT_FILE, "wb") as f:
            pickle.dump(all_results, f)

    except Exception as e:
        tqdm.write(f"[FAIL] {config_key}: {e}")

with open(SUMMARY_FILE, "w", encoding="utf-8") as f:
    for config, result in all_results.items():
        weight_path = result.get('weights_path', 'No information for weight')
        f.write(f"{config}: Weights saved at {weight_path}\n")


EfficientNetB0_case5_adam: 100%|██████████| 7/7 [6:15:34<00:00, 3219.21s/it, log=/home/hoai-linh.dao/Works/BraTS/results/self-experiments/cnns-Figshare-x10/logs/adam/EfficientNetB0_case5.txt]  
